In [1]:
# !pip install yahooquery

In [5]:
import pandas as pd
import pickle

from yahooquery import Ticker

# Class for Fetching Data and Calculating Piotroski Score

In [3]:
class StockFundamentals:
    def __init__(self, symbol):
        self.symbol = symbol
        self.ticker = Ticker(self.symbol)
        self.income_statement = None
        self.balance_sheet = None
        self.cash_flow = None

    def fetch_data(self):
        self.income_statement = self.ticker.income_statement(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
        self.balance_sheet = self.ticker.balance_sheet(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")
        self.cash_flow = self.ticker.cash_flow(frequency = 'q', trailing = False).sort_values('asOfDate').set_index("asOfDate")

class PiotroskiScoreCalculator:
    def __init__(self, stock):
        self.stock = stock
        self.date = None
        self.prev_date = None

    def set_dates(self, date):
        self.date = date
        self.prev_date = self.stock.balance_sheet.index[self.stock.balance_sheet.index.get_loc(date) - 1]

    def calculate_roa_score(self):
        roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
        return 1 if roa > 0 else 0

    def calculate_cfo_score(self):
        cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
        return 1 if cfo > 0 else 0

    def calculate_delta_roa_score(self):
        roa = self.stock.income_statement.loc[self.date, 'NetIncome'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
        delta_roa = roa - (self.stock.income_statement.loc[self.prev_date, 'NetIncome'] / \
                           self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets'])
        return 1 if delta_roa > 0 else 0

    def calculate_quality_of_earnings_score(self):
        cfo = self.stock.cash_flow.loc[self.date, 'OperatingCashFlow']
        return 1 if cfo > self.stock.income_statement.loc[self.date, 'NetIncome'] else 0

    def calculate_delta_leverage_score(self):
        delta_leverage = self.stock.balance_sheet.loc[self.date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets'] - \
                         self.stock.balance_sheet.loc[self.prev_date, 'LongTermDebt'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
        return 1 if delta_leverage < 0 else 0

    def calculate_delta_liquidity_score(self):
        delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \
                          (self.stock.balance_sheet.loc[self.prev_date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.prev_date, 'CurrentLiabilities'])
        return 1 if delta_liquidity > 0 else 0

    def calculate_new_equity_score(self):
        new_equity = self.stock.balance_sheet.loc[self.date, 'CommonStock'] - \
                     self.stock.balance_sheet.loc[self.prev_date, 'CommonStock']
        return 1 if new_equity <= 0 else 0

    def calculate_gross_margin_score(self):
        gross_margin_now = (self.stock.income_statement.loc[self.date, 'TotalRevenue'] - \
                            self.stock.income_statement.loc[self.date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.date, 'TotalRevenue']
        gross_margin_prev = (self.stock.income_statement.loc[self.prev_date, 'TotalRevenue'] - \
                             self.stock.income_statement.loc[self.prev_date, 'CostOfRevenue']) / self.stock.income_statement.loc[self.prev_date, 'TotalRevenue']
        return 1 if gross_margin_now > gross_margin_prev else 0

    def calculate_asset_turnover_score(self):
        asset_turnover_now = self.stock.income_statement.loc[self.date, 'TotalRevenue'] / self.stock.balance_sheet.loc[self.date, 'TotalAssets']
        asset_turnover_prev = self.stock.income_statement.loc[self.prev_date,
        'TotalRevenue'] / self.stock.balance_sheet.loc[self.prev_date, 'TotalAssets']
        return 1 if asset_turnover_now > asset_turnover_prev else 0

    def calculate_score(self):
        self.stock.fetch_data()
        score_data = []

        for date in self.stock.balance_sheet.index[1:]:
            self.set_dates(date)

            roa_score = self.calculate_roa_score()
            cfo_score = self.calculate_cfo_score()
            delta_roa_score = self.calculate_delta_roa_score()
            quality_of_earnings_score = self.calculate_quality_of_earnings_score()
            delta_leverage_score = self.calculate_delta_leverage_score()
            delta_liquidity_score = self.calculate_delta_liquidity_score()
            new_equity_score = self.calculate_new_equity_score()
            gross_margin_score = self.calculate_gross_margin_score()
            asset_turnover_score = self.calculate_asset_turnover_score()

            total_score = sum([roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, delta_leverage_score,
                                delta_liquidity_score, new_equity_score, gross_margin_score, asset_turnover_score])
            
            # Save the data
            score_data.append([date, roa_score, cfo_score, delta_roa_score, quality_of_earnings_score, 
                               delta_leverage_score, delta_liquidity_score, new_equity_score, 
                               gross_margin_score, asset_turnover_score, total_score])

        # Convert the data into a DataFrame
        scores_df = pd.DataFrame(score_data, columns=['Date', 'ROA', 'CFO', 'Delta ROA', 'Quality of Earnings', 
                                                      'Delta Leverage', 'Delta Liquidity', 'New Equity', 
                                                      'Gross Margin', 'Asset Turnover', 'Piotroski Score'])
        scores_df.set_index('Date', inplace=True)

        return scores_df


# Load Data

In [7]:
with open('companies.pkl', 'rb') as file:
    df_companies = pickle.load(file)

df_companies.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Calculate Piotroski Score

In [12]:

# Use the classes
stocks = df_companies.Symbol  # Add any stocks you're interested in
all_scores = []

i = 0
n = len(stocks)
for symbol in stocks:
    i += 1
    try:
        stock = StockFundamentals(symbol)
        calculator = PiotroskiScoreCalculator(stock)
        scores = calculator.calculate_score()
        scores['Ticker'] = symbol  # add a column for the ticker
        all_scores.append(scores)
    except Exception as e:
        print(f'Couldnt fetch data for {symbol} with error {e}')
        continue
    
    print(f"Iteration {i} of {n}")

# Concatenate all the individual DataFrames together
all_scores_df = pd.concat(all_scores)

all_scores_df

Iteration 1 of 503
Iteration 2 of 503
Iteration 3 of 503
Iteration 4 of 503
Iteration 5 of 503
Iteration 6 of 503
Iteration 7 of 503
Iteration 8 of 503
Iteration 9 of 503
Couldnt fetch data for AFL with error 'CurrentAssets'
Iteration 11 of 503
Iteration 12 of 503
Iteration 13 of 503
Iteration 14 of 503
Iteration 15 of 503
Iteration 16 of 503
Iteration 17 of 503
Couldnt fetch data for ALGN with error 'LongTermDebt'
Iteration 19 of 503
Iteration 20 of 503
Couldnt fetch data for ALL with error 'CurrentAssets'
Iteration 22 of 503
Iteration 23 of 503
Iteration 24 of 503
Iteration 25 of 503
Iteration 26 of 503
Iteration 27 of 503
Iteration 28 of 503
Iteration 29 of 503
Iteration 30 of 503
Couldnt fetch data for AXP with error 'CurrentAssets'
Couldnt fetch data for AIG with error 'CurrentAssets'
Iteration 33 of 503
Iteration 34 of 503
Couldnt fetch data for AMP with error 'CurrentAssets'
Iteration 36 of 503
Iteration 37 of 503
Iteration 38 of 503
Iteration 39 of 503
Iteration 40 of 503
Itera

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19140\624463908.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19140\624463908.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  (self.stock.balance_sheet.loc[self.prev_date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.prev_date, 'CurrentLiabilities'])
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19140\624463908.py:48: RuntimeWarning: invalid value encountered in scalar subtract
  delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \


Iteration 54 of 503
Iteration 55 of 503
Iteration 56 of 503
Iteration 57 of 503
Iteration 58 of 503
Couldnt fetch data for BAC with error 'CurrentAssets'
Iteration 60 of 503
Iteration 61 of 503
Iteration 62 of 503
Couldnt fetch data for WRB with error 'CurrentAssets'
Couldnt fetch data for BRK.B with error 'str' object has no attribute 'sort_values'
Iteration 65 of 503
Iteration 66 of 503
Iteration 67 of 503
Iteration 68 of 503
Iteration 69 of 503
Couldnt fetch data for BX with error 'LongTermDebt'
Couldnt fetch data for BK with error 'CurrentAssets'
Iteration 72 of 503
Couldnt fetch data for BKNG with error 'CostOfRevenue'
Iteration 74 of 503
Iteration 75 of 503
Iteration 76 of 503
Iteration 77 of 503
Iteration 78 of 503
Iteration 79 of 503
Iteration 80 of 503
Couldnt fetch data for BF.B with error 'str' object has no attribute 'sort_values'
Iteration 82 of 503
Iteration 83 of 503
Iteration 84 of 503
Iteration 85 of 503
Iteration 86 of 503
Iteration 87 of 503
Couldnt fetch data for CO

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19140\624463908.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  delta_liquidity = (self.stock.balance_sheet.loc[self.date, 'CurrentAssets'] / self.stock.balance_sheet.loc[self.date, 'CurrentLiabilities']) - \


Iteration 181 of 503
Iteration 182 of 503
Iteration 183 of 503
Couldnt fetch data for EG with error 'CurrentAssets'
Iteration 185 of 503
Iteration 186 of 503
Iteration 187 of 503
Iteration 188 of 503
Couldnt fetch data for EXPD with error 'LongTermDebt'
Iteration 190 of 503
Iteration 191 of 503
Couldnt fetch data for FFIV with error 'LongTermDebt'
Iteration 193 of 503
Iteration 194 of 503
Iteration 195 of 503
Iteration 196 of 503
Iteration 197 of 503
Couldnt fetch data for FITB with error 'CurrentAssets'
Iteration 199 of 503
Iteration 200 of 503
Iteration 201 of 503
Iteration 202 of 503
Iteration 203 of 503
Iteration 204 of 503
Iteration 205 of 503
Iteration 206 of 503
Iteration 207 of 503
Couldnt fetch data for FOXA with error 'CostOfRevenue'
Couldnt fetch data for FOX with error 'CostOfRevenue'
Iteration 210 of 503
Iteration 211 of 503
Couldnt fetch data for GRMN with error 'LongTermDebt'
Iteration 213 of 503
Iteration 214 of 503
Iteration 215 of 503
Iteration 216 of 503
Iteration 21

,ROA,CFO,Delta ROA,Quality of Earnings,Delta Leverage,Delta Liquidity,New Equity,Gross Margin,Asset Turnover,Piotroski Score,Ticker
Date,,,,,,,,,,,
2022-12-31,1,1,0,1,0,0,1,0,0,4,MMM
2023-03-31,1,1,1,1,1,0,1,0,0,6,MMM
2023-06-30,0,1,0,1,1,1,1,1,0,6,MMM
2023-09-30,0,1,1,1,1,0,1,1,0,6,MMM
2022-12-31,0,1,0,1,0,1,1,1,1,6,AOS
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,1,1,0,1,1,1,1,0,1,7,ZBH
2023-09-30,1,1,0,1,1,1,0,0,0,5,ZBH
2022-12-31,1,1,0,1,0,1,1,0,0,5,ZTS


In [13]:
with open('piotroskiScores.pkl', 'wb') as file:
    pickle.dump(all_scores_df, file)